In [3]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *

In [ ]:
# the raw nc files for VZA were downloaded for 15 days, not per month as LST
# for easier managing, files will be merged here

files = sorted(getFilelist('/data/Aldhani/eoagritwin/et/Sentinel3/VZA/raw', '.nc'))
year = 2018
outPath = f'/data/Aldhani/eoagritwin/et/Sentinel3/VZA/monthly_tiff_values/{year}/'
os.makedirs(outPath, exist_ok=True) 
    

In [9]:
# get a subset of files for that year
yearFiles = [file for file in files if int(file.split('/')[-1].split('_')[-1][0:4]) == year]

In [10]:
for month in [f'{i:02d}' for i in range(1,13)]:
    print([yF for yF in yearFiles if month == yF.split('-')[1]])

['/data/Aldhani/eoagritwin/et/Sentinel3/VZA/raw/Germany_2018-01-01.nc', '/data/Aldhani/eoagritwin/et/Sentinel3/VZA/raw/Germany_2018-01-15.nc']
['/data/Aldhani/eoagritwin/et/Sentinel3/VZA/raw/Germany_2018-02-01.nc', '/data/Aldhani/eoagritwin/et/Sentinel3/VZA/raw/Germany_2018-02-15.nc']
['/data/Aldhani/eoagritwin/et/Sentinel3/VZA/raw/Germany_2018-03-01.nc', '/data/Aldhani/eoagritwin/et/Sentinel3/VZA/raw/Germany_2018-03-15.nc']
['/data/Aldhani/eoagritwin/et/Sentinel3/VZA/raw/Germany_2018-04-01.nc', '/data/Aldhani/eoagritwin/et/Sentinel3/VZA/raw/Germany_2018-04-15.nc']
['/data/Aldhani/eoagritwin/et/Sentinel3/VZA/raw/Germany_2018-05-01.nc', '/data/Aldhani/eoagritwin/et/Sentinel3/VZA/raw/Germany_2018-05-15.nc']
['/data/Aldhani/eoagritwin/et/Sentinel3/VZA/raw/Germany_2018-06-01.nc', '/data/Aldhani/eoagritwin/et/Sentinel3/VZA/raw/Germany_2018-06-15.nc']
['/data/Aldhani/eoagritwin/et/Sentinel3/VZA/raw/Germany_2018-07-01.nc', '/data/Aldhani/eoagritwin/et/Sentinel3/VZA/raw/Germany_2018-07-15.nc']

In [7]:
for month in [f'{i:02d}' for i in range(1,13)]:
    if month == '02':
        break
    monthL = []
    files = [yF for yF in yearFiles if month == yF.split('-')[1]]
    for file in files:
        monthL.append(getDataFromNC_VZA(file))
    block = np.dstack(monthL)

In [ ]:
for month in [f'{i:02d}' for i in range(1,13)]:
    files = [yF for yF in yearFiles if month == yF.split('-')[1]]
    monthL = []
    bnames = []
    for file in files:
        monthL.append(getDataFromNC_VZA(file))
        accDateTimes = getAllDatesS3(file)
        for l in range(len(accDateTimes)):
            bnames.append(str(accDateTimes[l].astype('datetime64[s]')))

    block = np.dstack(monthL)
    block2 = np.where(block > 45, 0, 1)
    block2[np.isnan(block)] = -5
    block3 = block2.astype(np.int8)

    exportNCarrayDerivatesInt(file, outPath, f'VZA_MASK_{year}_{month}.tif', bnames, block3, datType=gdal.GDT_Int8, numberOfBands=block.shape[2], noData=-5)